<a href="https://colab.research.google.com/github/athira-tech1/C-_-/blob/main/final3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install blake3
!pip install gradio
!pip install fpdf
!pip install ffmpeg-python

In [5]:
import gradio as gr
import cv2
import numpy as np
import hashlib
import blake3
import os
from fpdf import FPDF
import ffmpeg
from skimage.metrics import structural_similarity as ssim

def extract_frames(video_path, resize_factor=0.25, sample_rate=30):
    supported_formats = ['.mp4', '.avi', '.mkv', '.mov']
    ext = os.path.splitext(video_path)[1].lower()
    if ext not in supported_formats:
        raise ValueError(f"Unsupported video format: {ext}. Supported formats are: {', '.join(supported_formats)}")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Failed to open video: {video_path}")

    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    for i in range(0, frame_count, sample_rate):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
        frames.append((frame, i / fps))
    cap.release()
    return frames, fps

def extract_metadata(video_path):
    try:
        probe = ffmpeg.probe(video_path)
        video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
        return {
            'Codec': video_info.get('codec_name', 'Unknown'),
            'Resolution': f"{video_info.get('width', '?')}x{video_info.get('height', '?')}",
            'Bitrate': video_info.get('bit_rate', 'Unknown'),
            'Duration': video_info.get('duration', 'Unknown')
        }
    except Exception as e:
        return {'Error': str(e)}

def generate_hash(frame, algorithm='sha256'):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hasher = hashlib.sha256() if algorithm == 'sha256' else blake3.blake3()
    hasher.update(gray_frame.tobytes())
    return hasher.hexdigest()
from skimage.metrics import structural_similarity as ssim

def detect_visual_tampering(orig_frame, tamp_frame, frame_index):
    orig_gray = cv2.cvtColor(orig_frame, cv2.COLOR_BGR2GRAY)
    tamp_gray = cv2.cvtColor(tamp_frame, cv2.COLOR_BGR2GRAY)


    brightness_diff = abs(np.mean(orig_gray) - np.mean(tamp_gray))
    contrast_diff = abs(np.std(orig_gray) - np.std(tamp_gray))


    similarity_score = ssim(orig_gray, tamp_gray)


    edges_orig = cv2.Canny(orig_gray, 50, 150)
    edges_tamp = cv2.Canny(tamp_gray, 50, 150)
    edge_diff = np.sum(edges_orig != edges_tamp)


    orig_hsv = cv2.cvtColor(orig_frame, cv2.COLOR_BGR2HSV)
    tamp_hsv = cv2.cvtColor(tamp_frame, cv2.COLOR_BGR2HSV)
    orig_hist = cv2.calcHist([orig_hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    tamp_hist = cv2.calcHist([tamp_hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    hist_similarity = cv2.compareHist(orig_hist, tamp_hist, cv2.HISTCMP_CORREL)


    output_path = f"tampered_frame_{frame_index}.png"
    comparison_image = np.hstack((orig_frame, tamp_frame))
    cv2.imwrite(output_path, comparison_image)


    if brightness_diff > 10:
        return "Brightness Change Detected", output_path
    elif contrast_diff > 15:
        return "Contrast Adjustment Detected", output_path
    elif similarity_score < 0.85:
        return "Structural Alteration Detected", output_path
    elif edge_diff > 5000:
        return "Edge Manipulation Detected", output_path
    elif hist_similarity < 0.85:
        return "Color Modification Detected", output_path
    else:
        return "Other Frame Manipulation Detected", output_path



def compare_frames(orig_frame, tamp_frame, frame_index):
    orig_hash_sha256 = generate_hash(orig_frame, 'sha256')
    orig_hash_blake3 = generate_hash(orig_frame, 'blake3')
    tamp_hash_sha256 = generate_hash(tamp_frame, 'sha256')
    tamp_hash_blake3 = generate_hash(tamp_frame, 'blake3')

    if orig_hash_sha256 != tamp_hash_sha256 or orig_hash_blake3 != tamp_hash_blake3:
        return True, detect_visual_tampering(orig_frame, tamp_frame, frame_index)
    return False, ("", "")

def detect_tampering(original_video, tampered_video):
    original_metadata = extract_metadata(original_video.name)
    tampered_metadata = extract_metadata(tampered_video.name)
    header_result = check_header_tampering(original_video.name, tampered_video.name)
    orig_frames, fps1 = extract_frames(original_video.name)
    tamp_frames, fps2 = extract_frames(tampered_video.name)
    tampered_indices = []
    comparison_images = []

    if fps1 != fps2 or len(orig_frames) != len(tamp_frames):
        result = "Tampered - Frame Mismatch"
    else:
        for i, ((orig_frame, ts1), (tamp_frame, ts2)) in enumerate(zip(orig_frames, tamp_frames)):
            tampered, (tamper_type, image_path) = compare_frames(orig_frame, tamp_frame, i)
            if tampered:
                tampered_indices.append((i, ts1, tamper_type))
                comparison_images.append(image_path)
        result = "Tampered" if tampered_indices else "Not Tampered"

    report_path = generate_report(result, tampered_indices, comparison_images, header_result, original_metadata, tampered_metadata)
    return result, header_result, comparison_images, report_path

def check_header_tampering(original_video, tampered_video):
    orig_size = os.path.getsize(original_video)
    tamp_size = os.path.getsize(tampered_video)
    return "Tampered" if orig_size != tamp_size else "Not Tampered"

def generate_report(result, tampered_indices, comparison_images, header_result, original_metadata, tampered_metadata):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, "Video Integrity Report", ln=True, align='C')
    pdf.ln(10)

    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, f"Overall Tampering Status: {result}\nHeader Tampering Status: {header_result}")
    pdf.ln(5)

    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "Frame Manipulation Details:", ln=True)
    pdf.set_font("Arial", size=12)
    for index, timestamp, tamper_type in tampered_indices:
        pdf.cell(0, 8, f"Frame {index} (Timestamp: {timestamp:.2f}s) - {tamper_type}", ln=True)
    pdf.ln(5)

    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "Metadata Comparison:", ln=True)
    pdf.set_font("Arial", size=12)
    for key in original_metadata.keys():
        pdf.cell(0, 8, f"{key}: Original - {original_metadata[key]}, Tampered - {tampered_metadata[key]}", ln=True)
    pdf.ln(5)

    for img_path in comparison_images:
        pdf.add_page()
        pdf.image(img_path, x=10, y=30, w=180)

    report_path = "tampering_report.pdf"
    pdf.output(report_path)
    return report_path

demo = gr.Interface(
    fn=detect_tampering,
    inputs=[gr.File(label='Original Video'), gr.File(label='Tampered Video')],
    outputs=[gr.Textbox(label='Tampering Status'), gr.Textbox(label='Header Tampered Status'), gr.Gallery(label='Frame Comparison View'), gr.File(label='Download Report')],
    title='Video Integrity Verification Framework',
    description='Upload original and potentially tampered videos to verify integrity with detailed tampering classification, metadata analysis, and report generation.'
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a5dee5101ec396a30.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
